In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import KFold
from sklearn import preprocessing
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn import metrics
from mlxtend.feature_selection import ColumnSelector
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import collections

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
path = '/Users/charlie/Desktop/insurance/'

In [3]:
tr = pd.read_csv(path+ 'tr3.csv')
te = pd.read_csv(path+ 'te3.csv')
policy = pd.read_csv(path+ 'policy2.csv')
claim = pd.read_csv(path+ 'claim_0702.csv')

In [4]:
policy.Main_Insurance_Coverage_Group.value_counts()

車責    1253120
車損     318013
竊盜     176809
Name: Main_Insurance_Coverage_Group, dtype: int64

In [5]:
#policy.groupby(['Policy_Number',''])
#policy.loc[policy.Coverage_Deductible_if_applied == 0, 'Coverage_Deductible_if_applied']
policy["no_coverage_deductible"] = np.where(policy.Coverage_Deductible_if_applied == 0, 1, 0)
policy["withdrawl_policy"] = np.where(policy.Coverage_Deductible_if_applied < 0, 1, 0)

In [6]:
tmp = policy.groupby('Policy_Number', as_index=False)['withdrawl_policy'].sum()
tmp.columns = ['Policy_Number', 'withdrawl_policy_count']
tr = pd.merge(tr, tmp, on='Policy_Number', how='left')
te = pd.merge(te, tmp, on='Policy_Number', how='left')

In [7]:
tmp = policy.groupby('Policy_Number', as_index=False)['no_coverage_deductible'].sum()
tmp.columns = ['Policy_Number', 'no_coverage_deductible_count']
tr = pd.merge(tr, tmp, on='Policy_Number', how='left')
te = pd.merge(te, tmp, on='Policy_Number', how='left')

In [8]:
# Claim_Number
tmp = claim.groupby(['Policy_Number','Coverage'], as_index=False)['Claim_Number'].count()
tmp = tmp.pivot_table(values= 'Claim_Number', index='Policy_Number', columns= 'Coverage')
tmp.columns = ['claim_'+col+'_count' for col in tmp]
tmp['Policy_Number'] = tmp.index
tr = pd.merge(tr, tmp, on='Policy_Number', how='left')
te = pd.merge(te, tmp, on='Policy_Number', how='left')

In [9]:
# Deductible
tmp = claim.groupby(['Policy_Number','Coverage'], as_index=False)['Deductible'].sum()
tmp = tmp.pivot_table(values= 'Deductible', index='Policy_Number', columns= 'Coverage')
tmp.columns = ['claim_'+col+'_sum' for col in tmp]
tmp['Policy_Number'] = tmp.index
tr = pd.merge(tr, tmp, on='Policy_Number', how='left')
te = pd.merge(te, tmp, on='Policy_Number', how='left')

In [69]:
# Deductible
tmp = claim.groupby(['Policy_Number','Coverage'], as_index=False)['Deductible'].sum()
tmp.columns = ["Policy_Number","Insurance_Coverage","Coverage_Claim_Deductible"]
policy = pd.merge(policy, tmp, on=['Policy_Number','Insurance_Coverage'], how='left')

In [83]:
#tmp = policy.groupby('Policy_Number', as_index=0).agg({"Coverage_Claim_Deductible":['sum','mean']})
#tmp.columns = ["Policy_Number","sum_Coverage_Claim_Deductible","mean_Coverage_Claim_Deductible"]
#tmp

In [16]:
def generate_claim_features(colname):
    global tr, te
    tmp = claim.groupby(['Policy_Number','Coverage'], as_index=0)[colname].sum()
    tmp = tmp.pivot_table(values= colname, index='Policy_Number', columns= 'Coverage')
    tmp.columns = ['claim_'+colname + '_'+ col+'_sum' for col in tmp]
    tmp['Policy_Number'] = tmp.index
    tr = pd.merge(tr, tmp, on='Policy_Number', how='left')
    te = pd.merge(te, tmp, on='Policy_Number', how='left')

In [17]:
generate_claim_features('Paid_Loss_Amount')
generate_claim_features('paid_Expenses_Amount')
generate_claim_features('Salvage_or_Subrogation?')
generate_claim_features('At_Fault?')
generate_claim_features('number_of_claimants')

In [18]:
from sklearn.decomposition import PCA
print tr.columns.values

['Policy_Number' 'Next_Premium' 'mean_Premium' ...,
 'claim_number_of_claimants_66L_sum' 'claim_number_of_claimants_70G_sum'
 'claim_number_of_claimants_70P_sum']


In [19]:
tmp = tr[['claim_00I_sum', 'claim_01A_sum', 'claim_02K_sum', 'claim_03L_sum',
       'claim_04M_sum', 'claim_05E_sum', 'claim_05N_sum', 'claim_06F_sum',
       'claim_07P_sum', 'claim_08H_sum', 'claim_09@_sum', 'claim_10A_sum',
       'claim_12L_sum', 'claim_14E_sum', 'claim_14N_sum', 'claim_15F_sum',
       'claim_15O_sum', 'claim_16G_sum', 'claim_16P_sum', 'claim_18@_sum',
       'claim_18I_sum', 'claim_20B_sum', 'claim_20K_sum', 'claim_25G_sum',
       'claim_26H_sum', 'claim_29B_sum', 'claim_29K_sum', 'claim_32N_sum',
       'claim_33F_sum', 'claim_33O_sum', 'claim_34P_sum', 'claim_35H_sum',
       'claim_36I_sum', 'claim_40M_sum', 'claim_41E_sum', 'claim_41N_sum',
       'claim_42F_sum', 'claim_45@_sum', 'claim_46A_sum', 'claim_51O_sum',
       'claim_55J_sum', 'claim_56B_sum', 'claim_56K_sum', 'claim_57C_sum',
       'claim_57L_sum', 'claim_66L_sum', 'claim_70G_sum', 'claim_70P_sum',
       'claim_Paid_Loss_Amount_00I_sum', 'claim_Paid_Loss_Amount_01A_sum',
       'claim_Paid_Loss_Amount_02K_sum', 'claim_Paid_Loss_Amount_03L_sum',
       'claim_Paid_Loss_Amount_04M_sum', 'claim_Paid_Loss_Amount_05E_sum',
       'claim_Paid_Loss_Amount_05N_sum', 'claim_Paid_Loss_Amount_06F_sum',
       'claim_Paid_Loss_Amount_07P_sum', 'claim_Paid_Loss_Amount_08H_sum',
       'claim_Paid_Loss_Amount_09@_sum', 'claim_Paid_Loss_Amount_10A_sum',
       'claim_Paid_Loss_Amount_12L_sum', 'claim_Paid_Loss_Amount_14E_sum',
       'claim_Paid_Loss_Amount_14N_sum', 'claim_Paid_Loss_Amount_15F_sum',
       'claim_Paid_Loss_Amount_15O_sum', 'claim_Paid_Loss_Amount_16G_sum',
       'claim_Paid_Loss_Amount_16P_sum', 'claim_Paid_Loss_Amount_18@_sum',
       'claim_Paid_Loss_Amount_18I_sum', 'claim_Paid_Loss_Amount_20B_sum',
       'claim_Paid_Loss_Amount_20K_sum', 'claim_Paid_Loss_Amount_25G_sum',
       'claim_Paid_Loss_Amount_26H_sum', 'claim_Paid_Loss_Amount_29B_sum',
       'claim_Paid_Loss_Amount_29K_sum', 'claim_Paid_Loss_Amount_32N_sum',
       'claim_Paid_Loss_Amount_33F_sum', 'claim_Paid_Loss_Amount_33O_sum',
       'claim_Paid_Loss_Amount_34P_sum', 'claim_Paid_Loss_Amount_35H_sum',
       'claim_Paid_Loss_Amount_36I_sum', 'claim_Paid_Loss_Amount_40M_sum',
       'claim_Paid_Loss_Amount_41E_sum', 'claim_Paid_Loss_Amount_41N_sum',
       'claim_Paid_Loss_Amount_42F_sum', 'claim_Paid_Loss_Amount_45@_sum',
       'claim_Paid_Loss_Amount_46A_sum', 'claim_Paid_Loss_Amount_51O_sum',
       'claim_Paid_Loss_Amount_55J_sum', 'claim_Paid_Loss_Amount_56B_sum',
       'claim_Paid_Loss_Amount_56K_sum', 'claim_Paid_Loss_Amount_57C_sum',
       'claim_Paid_Loss_Amount_57L_sum', 'claim_Paid_Loss_Amount_66L_sum',
       'claim_Paid_Loss_Amount_70G_sum', 'claim_Paid_Loss_Amount_70P_sum',
       'claim_paid_Expenses_Amount_00I_sum',
       'claim_paid_Expenses_Amount_01A_sum',
       'claim_paid_Expenses_Amount_02K_sum',
       'claim_paid_Expenses_Amount_03L_sum',
       'claim_paid_Expenses_Amount_04M_sum',
       'claim_paid_Expenses_Amount_05E_sum',
       'claim_paid_Expenses_Amount_05N_sum',
       'claim_paid_Expenses_Amount_06F_sum',
       'claim_paid_Expenses_Amount_07P_sum',
       'claim_paid_Expenses_Amount_08H_sum',
       'claim_paid_Expenses_Amount_09@_sum',
       'claim_paid_Expenses_Amount_10A_sum',
       'claim_paid_Expenses_Amount_12L_sum',
       'claim_paid_Expenses_Amount_14E_sum',
       'claim_paid_Expenses_Amount_14N_sum',
       'claim_paid_Expenses_Amount_15F_sum',
       'claim_paid_Expenses_Amount_15O_sum',
       'claim_paid_Expenses_Amount_16G_sum',
       'claim_paid_Expenses_Amount_16P_sum',
       'claim_paid_Expenses_Amount_18@_sum',
       'claim_paid_Expenses_Amount_18I_sum',
       'claim_paid_Expenses_Amount_20B_sum',
       'claim_paid_Expenses_Amount_20K_sum',
       'claim_paid_Expenses_Amount_25G_sum',
       'claim_paid_Expenses_Amount_26H_sum',
       'claim_paid_Expenses_Amount_29B_sum',
       'claim_paid_Expenses_Amount_29K_sum',
       'claim_paid_Expenses_Amount_32N_sum',
       'claim_paid_Expenses_Amount_33F_sum',
       'claim_paid_Expenses_Amount_33O_sum',
       'claim_paid_Expenses_Amount_34P_sum',
       'claim_paid_Expenses_Amount_35H_sum',
       'claim_paid_Expenses_Amount_36I_sum',
       'claim_paid_Expenses_Amount_40M_sum',
       'claim_paid_Expenses_Amount_41E_sum',
       'claim_paid_Expenses_Amount_41N_sum',
       'claim_paid_Expenses_Amount_42F_sum',
       'claim_paid_Expenses_Amount_45@_sum',
       'claim_paid_Expenses_Amount_46A_sum',
       'claim_paid_Expenses_Amount_51O_sum',
       'claim_paid_Expenses_Amount_55J_sum',
       'claim_paid_Expenses_Amount_56B_sum',
       'claim_paid_Expenses_Amount_56K_sum',
       'claim_paid_Expenses_Amount_57C_sum',
       'claim_paid_Expenses_Amount_57L_sum',
       'claim_paid_Expenses_Amount_66L_sum',
       'claim_paid_Expenses_Amount_70G_sum',
       'claim_paid_Expenses_Amount_70P_sum',
       'claim_Salvage_or_Subrogation?_00I_sum',
       'claim_Salvage_or_Subrogation?_01A_sum',
       'claim_Salvage_or_Subrogation?_02K_sum',
       'claim_Salvage_or_Subrogation?_03L_sum',
       'claim_Salvage_or_Subrogation?_04M_sum',
       'claim_Salvage_or_Subrogation?_05E_sum',
       'claim_Salvage_or_Subrogation?_05N_sum',
       'claim_Salvage_or_Subrogation?_06F_sum',
       'claim_Salvage_or_Subrogation?_07P_sum',
       'claim_Salvage_or_Subrogation?_08H_sum',
       'claim_Salvage_or_Subrogation?_09@_sum',
       'claim_Salvage_or_Subrogation?_10A_sum',
       'claim_Salvage_or_Subrogation?_12L_sum',
       'claim_Salvage_or_Subrogation?_14E_sum',
       'claim_Salvage_or_Subrogation?_14N_sum',
       'claim_Salvage_or_Subrogation?_15F_sum',
       'claim_Salvage_or_Subrogation?_15O_sum',
       'claim_Salvage_or_Subrogation?_16G_sum',
       'claim_Salvage_or_Subrogation?_16P_sum',
       'claim_Salvage_or_Subrogation?_18@_sum',
       'claim_Salvage_or_Subrogation?_18I_sum',
       'claim_Salvage_or_Subrogation?_20B_sum',
       'claim_Salvage_or_Subrogation?_20K_sum',
       'claim_Salvage_or_Subrogation?_25G_sum',
       'claim_Salvage_or_Subrogation?_26H_sum',
       'claim_Salvage_or_Subrogation?_29B_sum',
       'claim_Salvage_or_Subrogation?_29K_sum',
       'claim_Salvage_or_Subrogation?_32N_sum',
       'claim_Salvage_or_Subrogation?_33F_sum',
       'claim_Salvage_or_Subrogation?_33O_sum',
       'claim_Salvage_or_Subrogation?_34P_sum',
       'claim_Salvage_or_Subrogation?_35H_sum',
       'claim_Salvage_or_Subrogation?_36I_sum',
       'claim_Salvage_or_Subrogation?_40M_sum',
       'claim_Salvage_or_Subrogation?_41E_sum',
       'claim_Salvage_or_Subrogation?_41N_sum',
       'claim_Salvage_or_Subrogation?_42F_sum',
       'claim_Salvage_or_Subrogation?_45@_sum',
       'claim_Salvage_or_Subrogation?_46A_sum',
       'claim_Salvage_or_Subrogation?_51O_sum',
       'claim_Salvage_or_Subrogation?_55J_sum',
       'claim_Salvage_or_Subrogation?_56B_sum',
       'claim_Salvage_or_Subrogation?_56K_sum',
       'claim_Salvage_or_Subrogation?_57C_sum',
       'claim_Salvage_or_Subrogation?_57L_sum',
       'claim_Salvage_or_Subrogation?_66L_sum',
       'claim_Salvage_or_Subrogation?_70G_sum',
       'claim_Salvage_or_Subrogation?_70P_sum', 'claim_At_Fault?_00I_sum',
       'claim_At_Fault?_01A_sum', 'claim_At_Fault?_02K_sum',
       'claim_At_Fault?_03L_sum', 'claim_At_Fault?_04M_sum',
       'claim_At_Fault?_05E_sum', 'claim_At_Fault?_05N_sum',
       'claim_At_Fault?_06F_sum', 'claim_At_Fault?_07P_sum',
       'claim_At_Fault?_08H_sum', 'claim_At_Fault?_09@_sum',
       'claim_At_Fault?_10A_sum', 'claim_At_Fault?_12L_sum',
       'claim_At_Fault?_14E_sum', 'claim_At_Fault?_14N_sum',
       'claim_At_Fault?_15F_sum', 'claim_At_Fault?_15O_sum',
       'claim_At_Fault?_16G_sum', 'claim_At_Fault?_16P_sum',
       'claim_At_Fault?_18@_sum', 'claim_At_Fault?_18I_sum',
       'claim_At_Fault?_20B_sum', 'claim_At_Fault?_20K_sum',
       'claim_At_Fault?_25G_sum', 'claim_At_Fault?_26H_sum',
       'claim_At_Fault?_29B_sum', 'claim_At_Fault?_29K_sum',
       'claim_At_Fault?_32N_sum', 'claim_At_Fault?_33F_sum',
       'claim_At_Fault?_33O_sum', 'claim_At_Fault?_34P_sum',
       'claim_At_Fault?_35H_sum', 'claim_At_Fault?_36I_sum',
       'claim_At_Fault?_40M_sum', 'claim_At_Fault?_41E_sum',
       'claim_At_Fault?_41N_sum', 'claim_At_Fault?_42F_sum',
       'claim_At_Fault?_45@_sum', 'claim_At_Fault?_46A_sum',
       'claim_At_Fault?_51O_sum', 'claim_At_Fault?_55J_sum',
       'claim_At_Fault?_56B_sum', 'claim_At_Fault?_56K_sum',
       'claim_At_Fault?_57C_sum', 'claim_At_Fault?_57L_sum',
       'claim_At_Fault?_66L_sum', 'claim_At_Fault?_70G_sum',
       'claim_At_Fault?_70P_sum', 'claim_number_of_claimants_00I_sum',
       'claim_number_of_claimants_01A_sum',
       'claim_number_of_claimants_02K_sum',
       'claim_number_of_claimants_03L_sum',
       'claim_number_of_claimants_04M_sum',
       'claim_number_of_claimants_05E_sum',
       'claim_number_of_claimants_05N_sum',
       'claim_number_of_claimants_06F_sum',
       'claim_number_of_claimants_07P_sum',
       'claim_number_of_claimants_08H_sum',
       'claim_number_of_claimants_09@_sum',
       'claim_number_of_claimants_10A_sum',
       'claim_number_of_claimants_12L_sum',
       'claim_number_of_claimants_14E_sum',
       'claim_number_of_claimants_14N_sum',
       'claim_number_of_claimants_15F_sum',
       'claim_number_of_claimants_15O_sum',
       'claim_number_of_claimants_16G_sum',
       'claim_number_of_claimants_16P_sum',
       'claim_number_of_claimants_18@_sum',
       'claim_number_of_claimants_18I_sum',
       'claim_number_of_claimants_20B_sum',
       'claim_number_of_claimants_20K_sum',
       'claim_number_of_claimants_25G_sum',
       'claim_number_of_claimants_26H_sum',
       'claim_number_of_claimants_29B_sum',
       'claim_number_of_claimants_29K_sum',
       'claim_number_of_claimants_32N_sum',
       'claim_number_of_claimants_33F_sum',
       'claim_number_of_claimants_33O_sum',
       'claim_number_of_claimants_34P_sum',
       'claim_number_of_claimants_35H_sum',
       'claim_number_of_claimants_36I_sum',
       'claim_number_of_claimants_40M_sum',
       'claim_number_of_claimants_41E_sum',
       'claim_number_of_claimants_41N_sum',
       'claim_number_of_claimants_42F_sum',
       'claim_number_of_claimants_45@_sum',
       'claim_number_of_claimants_46A_sum',
       'claim_number_of_claimants_51O_sum',
       'claim_number_of_claimants_55J_sum',
       'claim_number_of_claimants_56B_sum',
       'claim_number_of_claimants_56K_sum',
       'claim_number_of_claimants_57C_sum',
       'claim_number_of_claimants_57L_sum',
       'claim_number_of_claimants_66L_sum',
       'claim_number_of_claimants_70G_sum',
       'claim_number_of_claimants_70P_sum']]
tmp2 = te[['claim_00I_sum', 'claim_01A_sum', 'claim_02K_sum', 'claim_03L_sum',
       'claim_04M_sum', 'claim_05E_sum', 'claim_05N_sum', 'claim_06F_sum',
       'claim_07P_sum', 'claim_08H_sum', 'claim_09@_sum', 'claim_10A_sum',
       'claim_12L_sum', 'claim_14E_sum', 'claim_14N_sum', 'claim_15F_sum',
       'claim_15O_sum', 'claim_16G_sum', 'claim_16P_sum', 'claim_18@_sum',
       'claim_18I_sum', 'claim_20B_sum', 'claim_20K_sum', 'claim_25G_sum',
       'claim_26H_sum', 'claim_29B_sum', 'claim_29K_sum', 'claim_32N_sum',
       'claim_33F_sum', 'claim_33O_sum', 'claim_34P_sum', 'claim_35H_sum',
       'claim_36I_sum', 'claim_40M_sum', 'claim_41E_sum', 'claim_41N_sum',
       'claim_42F_sum', 'claim_45@_sum', 'claim_46A_sum', 'claim_51O_sum',
       'claim_55J_sum', 'claim_56B_sum', 'claim_56K_sum', 'claim_57C_sum',
       'claim_57L_sum', 'claim_66L_sum', 'claim_70G_sum', 'claim_70P_sum',
       'claim_Paid_Loss_Amount_00I_sum', 'claim_Paid_Loss_Amount_01A_sum',
       'claim_Paid_Loss_Amount_02K_sum', 'claim_Paid_Loss_Amount_03L_sum',
       'claim_Paid_Loss_Amount_04M_sum', 'claim_Paid_Loss_Amount_05E_sum',
       'claim_Paid_Loss_Amount_05N_sum', 'claim_Paid_Loss_Amount_06F_sum',
       'claim_Paid_Loss_Amount_07P_sum', 'claim_Paid_Loss_Amount_08H_sum',
       'claim_Paid_Loss_Amount_09@_sum', 'claim_Paid_Loss_Amount_10A_sum',
       'claim_Paid_Loss_Amount_12L_sum', 'claim_Paid_Loss_Amount_14E_sum',
       'claim_Paid_Loss_Amount_14N_sum', 'claim_Paid_Loss_Amount_15F_sum',
       'claim_Paid_Loss_Amount_15O_sum', 'claim_Paid_Loss_Amount_16G_sum',
       'claim_Paid_Loss_Amount_16P_sum', 'claim_Paid_Loss_Amount_18@_sum',
       'claim_Paid_Loss_Amount_18I_sum', 'claim_Paid_Loss_Amount_20B_sum',
       'claim_Paid_Loss_Amount_20K_sum', 'claim_Paid_Loss_Amount_25G_sum',
       'claim_Paid_Loss_Amount_26H_sum', 'claim_Paid_Loss_Amount_29B_sum',
       'claim_Paid_Loss_Amount_29K_sum', 'claim_Paid_Loss_Amount_32N_sum',
       'claim_Paid_Loss_Amount_33F_sum', 'claim_Paid_Loss_Amount_33O_sum',
       'claim_Paid_Loss_Amount_34P_sum', 'claim_Paid_Loss_Amount_35H_sum',
       'claim_Paid_Loss_Amount_36I_sum', 'claim_Paid_Loss_Amount_40M_sum',
       'claim_Paid_Loss_Amount_41E_sum', 'claim_Paid_Loss_Amount_41N_sum',
       'claim_Paid_Loss_Amount_42F_sum', 'claim_Paid_Loss_Amount_45@_sum',
       'claim_Paid_Loss_Amount_46A_sum', 'claim_Paid_Loss_Amount_51O_sum',
       'claim_Paid_Loss_Amount_55J_sum', 'claim_Paid_Loss_Amount_56B_sum',
       'claim_Paid_Loss_Amount_56K_sum', 'claim_Paid_Loss_Amount_57C_sum',
       'claim_Paid_Loss_Amount_57L_sum', 'claim_Paid_Loss_Amount_66L_sum',
       'claim_Paid_Loss_Amount_70G_sum', 'claim_Paid_Loss_Amount_70P_sum',
       'claim_paid_Expenses_Amount_00I_sum',
       'claim_paid_Expenses_Amount_01A_sum',
       'claim_paid_Expenses_Amount_02K_sum',
       'claim_paid_Expenses_Amount_03L_sum',
       'claim_paid_Expenses_Amount_04M_sum',
       'claim_paid_Expenses_Amount_05E_sum',
       'claim_paid_Expenses_Amount_05N_sum',
       'claim_paid_Expenses_Amount_06F_sum',
       'claim_paid_Expenses_Amount_07P_sum',
       'claim_paid_Expenses_Amount_08H_sum',
       'claim_paid_Expenses_Amount_09@_sum',
       'claim_paid_Expenses_Amount_10A_sum',
       'claim_paid_Expenses_Amount_12L_sum',
       'claim_paid_Expenses_Amount_14E_sum',
       'claim_paid_Expenses_Amount_14N_sum',
       'claim_paid_Expenses_Amount_15F_sum',
       'claim_paid_Expenses_Amount_15O_sum',
       'claim_paid_Expenses_Amount_16G_sum',
       'claim_paid_Expenses_Amount_16P_sum',
       'claim_paid_Expenses_Amount_18@_sum',
       'claim_paid_Expenses_Amount_18I_sum',
       'claim_paid_Expenses_Amount_20B_sum',
       'claim_paid_Expenses_Amount_20K_sum',
       'claim_paid_Expenses_Amount_25G_sum',
       'claim_paid_Expenses_Amount_26H_sum',
       'claim_paid_Expenses_Amount_29B_sum',
       'claim_paid_Expenses_Amount_29K_sum',
       'claim_paid_Expenses_Amount_32N_sum',
       'claim_paid_Expenses_Amount_33F_sum',
       'claim_paid_Expenses_Amount_33O_sum',
       'claim_paid_Expenses_Amount_34P_sum',
       'claim_paid_Expenses_Amount_35H_sum',
       'claim_paid_Expenses_Amount_36I_sum',
       'claim_paid_Expenses_Amount_40M_sum',
       'claim_paid_Expenses_Amount_41E_sum',
       'claim_paid_Expenses_Amount_41N_sum',
       'claim_paid_Expenses_Amount_42F_sum',
       'claim_paid_Expenses_Amount_45@_sum',
       'claim_paid_Expenses_Amount_46A_sum',
       'claim_paid_Expenses_Amount_51O_sum',
       'claim_paid_Expenses_Amount_55J_sum',
       'claim_paid_Expenses_Amount_56B_sum',
       'claim_paid_Expenses_Amount_56K_sum',
       'claim_paid_Expenses_Amount_57C_sum',
       'claim_paid_Expenses_Amount_57L_sum',
       'claim_paid_Expenses_Amount_66L_sum',
       'claim_paid_Expenses_Amount_70G_sum',
       'claim_paid_Expenses_Amount_70P_sum',
       'claim_Salvage_or_Subrogation?_00I_sum',
       'claim_Salvage_or_Subrogation?_01A_sum',
       'claim_Salvage_or_Subrogation?_02K_sum',
       'claim_Salvage_or_Subrogation?_03L_sum',
       'claim_Salvage_or_Subrogation?_04M_sum',
       'claim_Salvage_or_Subrogation?_05E_sum',
       'claim_Salvage_or_Subrogation?_05N_sum',
       'claim_Salvage_or_Subrogation?_06F_sum',
       'claim_Salvage_or_Subrogation?_07P_sum',
       'claim_Salvage_or_Subrogation?_08H_sum',
       'claim_Salvage_or_Subrogation?_09@_sum',
       'claim_Salvage_or_Subrogation?_10A_sum',
       'claim_Salvage_or_Subrogation?_12L_sum',
       'claim_Salvage_or_Subrogation?_14E_sum',
       'claim_Salvage_or_Subrogation?_14N_sum',
       'claim_Salvage_or_Subrogation?_15F_sum',
       'claim_Salvage_or_Subrogation?_15O_sum',
       'claim_Salvage_or_Subrogation?_16G_sum',
       'claim_Salvage_or_Subrogation?_16P_sum',
       'claim_Salvage_or_Subrogation?_18@_sum',
       'claim_Salvage_or_Subrogation?_18I_sum',
       'claim_Salvage_or_Subrogation?_20B_sum',
       'claim_Salvage_or_Subrogation?_20K_sum',
       'claim_Salvage_or_Subrogation?_25G_sum',
       'claim_Salvage_or_Subrogation?_26H_sum',
       'claim_Salvage_or_Subrogation?_29B_sum',
       'claim_Salvage_or_Subrogation?_29K_sum',
       'claim_Salvage_or_Subrogation?_32N_sum',
       'claim_Salvage_or_Subrogation?_33F_sum',
       'claim_Salvage_or_Subrogation?_33O_sum',
       'claim_Salvage_or_Subrogation?_34P_sum',
       'claim_Salvage_or_Subrogation?_35H_sum',
       'claim_Salvage_or_Subrogation?_36I_sum',
       'claim_Salvage_or_Subrogation?_40M_sum',
       'claim_Salvage_or_Subrogation?_41E_sum',
       'claim_Salvage_or_Subrogation?_41N_sum',
       'claim_Salvage_or_Subrogation?_42F_sum',
       'claim_Salvage_or_Subrogation?_45@_sum',
       'claim_Salvage_or_Subrogation?_46A_sum',
       'claim_Salvage_or_Subrogation?_51O_sum',
       'claim_Salvage_or_Subrogation?_55J_sum',
       'claim_Salvage_or_Subrogation?_56B_sum',
       'claim_Salvage_or_Subrogation?_56K_sum',
       'claim_Salvage_or_Subrogation?_57C_sum',
       'claim_Salvage_or_Subrogation?_57L_sum',
       'claim_Salvage_or_Subrogation?_66L_sum',
       'claim_Salvage_or_Subrogation?_70G_sum',
       'claim_Salvage_or_Subrogation?_70P_sum', 'claim_At_Fault?_00I_sum',
       'claim_At_Fault?_01A_sum', 'claim_At_Fault?_02K_sum',
       'claim_At_Fault?_03L_sum', 'claim_At_Fault?_04M_sum',
       'claim_At_Fault?_05E_sum', 'claim_At_Fault?_05N_sum',
       'claim_At_Fault?_06F_sum', 'claim_At_Fault?_07P_sum',
       'claim_At_Fault?_08H_sum', 'claim_At_Fault?_09@_sum',
       'claim_At_Fault?_10A_sum', 'claim_At_Fault?_12L_sum',
       'claim_At_Fault?_14E_sum', 'claim_At_Fault?_14N_sum',
       'claim_At_Fault?_15F_sum', 'claim_At_Fault?_15O_sum',
       'claim_At_Fault?_16G_sum', 'claim_At_Fault?_16P_sum',
       'claim_At_Fault?_18@_sum', 'claim_At_Fault?_18I_sum',
       'claim_At_Fault?_20B_sum', 'claim_At_Fault?_20K_sum',
       'claim_At_Fault?_25G_sum', 'claim_At_Fault?_26H_sum',
       'claim_At_Fault?_29B_sum', 'claim_At_Fault?_29K_sum',
       'claim_At_Fault?_32N_sum', 'claim_At_Fault?_33F_sum',
       'claim_At_Fault?_33O_sum', 'claim_At_Fault?_34P_sum',
       'claim_At_Fault?_35H_sum', 'claim_At_Fault?_36I_sum',
       'claim_At_Fault?_40M_sum', 'claim_At_Fault?_41E_sum',
       'claim_At_Fault?_41N_sum', 'claim_At_Fault?_42F_sum',
       'claim_At_Fault?_45@_sum', 'claim_At_Fault?_46A_sum',
       'claim_At_Fault?_51O_sum', 'claim_At_Fault?_55J_sum',
       'claim_At_Fault?_56B_sum', 'claim_At_Fault?_56K_sum',
       'claim_At_Fault?_57C_sum', 'claim_At_Fault?_57L_sum',
       'claim_At_Fault?_66L_sum', 'claim_At_Fault?_70G_sum',
       'claim_At_Fault?_70P_sum', 'claim_number_of_claimants_00I_sum',
       'claim_number_of_claimants_01A_sum',
       'claim_number_of_claimants_02K_sum',
       'claim_number_of_claimants_03L_sum',
       'claim_number_of_claimants_04M_sum',
       'claim_number_of_claimants_05E_sum',
       'claim_number_of_claimants_05N_sum',
       'claim_number_of_claimants_06F_sum',
       'claim_number_of_claimants_07P_sum',
       'claim_number_of_claimants_08H_sum',
       'claim_number_of_claimants_09@_sum',
       'claim_number_of_claimants_10A_sum',
       'claim_number_of_claimants_12L_sum',
       'claim_number_of_claimants_14E_sum',
       'claim_number_of_claimants_14N_sum',
       'claim_number_of_claimants_15F_sum',
       'claim_number_of_claimants_15O_sum',
       'claim_number_of_claimants_16G_sum',
       'claim_number_of_claimants_16P_sum',
       'claim_number_of_claimants_18@_sum',
       'claim_number_of_claimants_18I_sum',
       'claim_number_of_claimants_20B_sum',
       'claim_number_of_claimants_20K_sum',
       'claim_number_of_claimants_25G_sum',
       'claim_number_of_claimants_26H_sum',
       'claim_number_of_claimants_29B_sum',
       'claim_number_of_claimants_29K_sum',
       'claim_number_of_claimants_32N_sum',
       'claim_number_of_claimants_33F_sum',
       'claim_number_of_claimants_33O_sum',
       'claim_number_of_claimants_34P_sum',
       'claim_number_of_claimants_35H_sum',
       'claim_number_of_claimants_36I_sum',
       'claim_number_of_claimants_40M_sum',
       'claim_number_of_claimants_41E_sum',
       'claim_number_of_claimants_41N_sum',
       'claim_number_of_claimants_42F_sum',
       'claim_number_of_claimants_45@_sum',
       'claim_number_of_claimants_46A_sum',
       'claim_number_of_claimants_51O_sum',
       'claim_number_of_claimants_55J_sum',
       'claim_number_of_claimants_56B_sum',
       'claim_number_of_claimants_56K_sum',
       'claim_number_of_claimants_57C_sum',
       'claim_number_of_claimants_57L_sum',
       'claim_number_of_claimants_66L_sum',
       'claim_number_of_claimants_70G_sum',
       'claim_number_of_claimants_70P_sum']]
tmp = tmp.fillna(0)
tmp2 = tmp2.fillna(0)
pca=PCA(n_components=15)
tr_Coverage_pca = pca.fit_transform(tmp)
te_Coverage_pca = pca.transform(tmp2)
sum(pca.explained_variance_ratio_)

0.99192435298262183

In [20]:
tmp = pd.DataFrame(tr_Coverage_pca)
tmp.columns = ['claim_coverage_pca_' + str(i) for i in range(15)]
tr = pd.concat([tr, tmp], axis=1)

tmp2 = pd.DataFrame(te_Coverage_pca)
tmp2.columns = ['claim_coverage_pca_' + str(i) for i in range(15)]
te = pd.concat([te, tmp2], axis=1)

In [21]:
print len(tr)
print len(te)

210763
140510


In [46]:
tmp = claim.groupby(['Policy_Number','Coverage'], as_index=False)['Claim_Number'].count()
tmp.columns = ["Policy_Number","Insurance_Coverage","Coverage_Claim_Number"]
policy = pd.merge(policy, tmp, on=['Policy_Number','Insurance_Coverage'], how='left')

In [47]:
policy.Coverage_Claim_Number.describe()

count    53769.000000
mean         1.294668
std          0.665863
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         15.000000
Name: Coverage_Claim_Number, dtype: float64

In [11]:
claim['no_fault'] = np.where(claim["At_Fault?"] == 0, 1, 0)

In [12]:
tmp = claim.groupby('Policy_Number', as_index=0)["no_fault"].sum()
tmp.columns = ["Policy_Number","no_fault_count"]
tr = pd.merge(tr, tmp, on='Policy_Number', how='left')
te = pd.merge(te, tmp, on='Policy_Number', how='left')

In [13]:
tr['no_fault_percentage'] = tr.no_fault_count / tr.Claim_Number
te['no_fault_percentage'] = te.no_fault_count / te.Claim_Number

In [38]:
policy.loc[policy.Insurance_Coverage == '56B','Coverage_Deductible_if_applied'].value_counts()

0    96
Name: Coverage_Deductible_if_applied, dtype: int64

In [22]:
tr.to_csv(path+'tr3.csv', index=False, encoding='utf-8')
te.to_csv(path+'te3.csv', index=False, encoding='utf-8')

In [14]:
print len(tr)
print len(te)

210763
140510
